<h3>機械学習課題 6 : Linear Regression 回帰分析 Part 4 - Option : Linear Regression with Multiple Variables</h3><br/>

<h4>注意 : pythonコードを実行しながら読んでください!</h4><br/>

今回の課題はオプションですが、簡単ですのでやってみるのをお勧めします。

課題5まで皆さんは

$$\hat{Y} = h(X) = \theta_0 + \theta_1 * X$$

のように、一つの変数$X$に対した仮定関数を作成してみました。これはすごく簡単なモデルで、実は世の中にはもっと複雑なケースがたくさん存在しています。前回はある都市の人口と年収との関係を推論モデル化してみましたが、例えば、

<h5>家の広さと部屋の数の二つの情報から家の値段を推論する。</h5>

みたいな問題を考えると、どうすれば良いでしょう。ちょっと仮定関数の形を変えてから回帰分析(Linear Regression)を利用して推論モデルを完成してみましょう。

$$\hat{Y} = h(X) = \theta_0 + \theta_1 * X_1 + \theta_2 * X_2$$

<br/>
まずはテスト用データがありますので、データの特性の理解の為に内容を確認してみます。

In [ ]:
#import required libraries
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

#read learning data X and Y
data = np.loadtxt('ex6data2.txt', delimiter=',')
X = data[:,0:2]
Y = data[:,2]
m = Y.size

df = pd.DataFrame(data,columns=['X1 : House Size in square feet', 'X2 : Room number', 'Y : Price in USD'])
df

データ自体は分かりやすく見えますが、一つ問題があります。

<h4>X1とX2のデータの範囲が全然違う問題です。（X1 : 825〜4478、X2 : 1〜5）</h4>

通常、複数のパラメーターのインプットデータを学習させる場合に、各パラメーターの範囲が異なるケースが多く、学習率(Learning Rate)の適用がパラメーター毎でバラバラになってしまいます。こうなると、最急降下法(Gradient Descent)で相当な時間がかかるなど、正しく動作しない可能性が高くなります。

この場合、良く使うのがスケーリング(Feature Scaling)という手法です。具体的には

$$newX_i = \frac{X_i - Xの平均}{Xの標準偏差}$$

の式でスケーリングというのを行います。スケーリングされたXの値はだいたい$-1 \leq X \leq 1$の範囲に収まります。

<h3>ここで課題です。インプットパラメーターXのスケーリングコードを作成してみましょう。</h3>

必要なライブラリは最初にimportしておきますので、使ってください。<br/>

<h4>コード作成後、実行すると、自動採点を行います。$Correct$が表示されると成功です。</h4>

<h4>注意：</h4>

- 以下の部分にコードを書いてください。

#------- Coding Start -------
#------- Coding End -------

- numpyに平均、標準偏差取得用のAPIがありますので、使ってください。

https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html

https://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html

- このあと、回帰分析を完成しますので、スケーリング有無でどう変わるか試してみてください。

In [ ]:
# Feature Scaling Function
def featureScaling(X):
    X_norm = X.copy()
    Xmean = np.zeros(X.shape[1])
    Xstd = np.zeros(X.shape[1])        
    
    #------- Coding Start -------

    #------- Coding End -------
    
    return X_norm, Xmean, Xstd

X, Xmean, Xstd = featureScaling(X)
#-----------------------------------------------------------
# Evaluation
#-----------------------------------------------------------
def evaluate(X, min, max):
    print("After Scaling = ", X, " => Correct") if X > min and X < max else print("After Scaling = ", X, " => Wrong")

evaluate(X[0,0], 0.131415, 0.131416) 
evaluate(X[15,1], 1.102205, 1.102206) 
evaluate(X[46,1], -0.226094, -0.226093) 

#-----------------------------------------------------------
# Print X after feature scaling
#-----------------------------------------------------------
df = pd.DataFrame(X,columns=['X1 : House Size in square feet with scaling', 'X2 : Room number with scaling'])
df

<h4>スケーリングについては以上です。続きます。</h4>

仮に、以下のように、もっとたくさんの情報を使う推論モデルも考えられると思います。

$$\hat{Y} = h(X) = \theta_0 + \theta_1 * X_1 + \theta_2 * X_2 + ... + \theta_n * X_n$$

ちなみに、これみたいは複数のパラメーターが存在する場合は、行列の形で考えた方がプログラミングの際にすごく楽になります。

$$\hat{Y} = h(X) = \begin{bmatrix} 
1 & X_1 & X_2 & ... & X_n
\end{bmatrix}
\begin{bmatrix} 
\theta_0 \\
\theta_1 \\
\theta_2 \\
... \\
\theta_n \\
\end{bmatrix}$$

ここでちょっと考えてみましょう。前回皆さんが作成してみた目的関数(Cost Function)を複数パラーメーター処理ができるように、行列を使った形にしてみましょう。（教師データが$m$個存在、あと一つの教師データに$n$個のパラメーターがあると想定します。）

<br/>
$$J = \frac{1}{2m}\sum_{i=1}^m(h(X^i) - Y^i)^2$$
<br/>

まず、

$$h(X) - Y = \begin{bmatrix} 
1 & X^1_1 & X^1_2 & ... & X^1_n \\
1 & X^2_1 & X^2_2 & ... & X^2_n \\
1 & X^3_1 & X^3_2 & ... & X^3_n \\
...  \\
1 & X^{m-1}_1 & X^{m-1}_2 & ... & X^{m-1}_n \\
1 & X^m_1 & X^m_2 & ... & X^m_n \\
\end{bmatrix}
\begin{bmatrix} 
\theta_0 \\
\theta_1 \\
\theta_2 \\
... \\
\theta_n \\ 
\end{bmatrix}
- \begin{bmatrix} 
Y_1 \\
Y_2 \\
... \\
Y_m \\ 
\end{bmatrix}$$

<br/><br/>
に表すことができました。$m$ x $n$回の掛算が簡単にできるようになりました。あと、$h(X) - Y$の2乗と$\sum$の計算が必要です。

<h3>ここで課題です。複数パラメーターの処理が可能な目的関数Cost Functionを作成してください。</h3>

必要なライブラリは最初にimportしておきましたので、使ってください。<br/>

<h4>コード作成後、実行すると、自動採点を行います。$Correct$が表示されると成功です。</h4>

<h4>注意：</h4>

- 以下の部分にコードを書いてください。

#------- Coding Start -------
#------- Coding End -------

- Loop利用禁止です。必ず行列演算のみ使ってください。
- numpyのdot関数利用をお進めします。（dot関数のみでも作成可能です）https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html

In [ ]:
X = np.c_[np.ones((m,1)),X]
Y = np.c_[Y]
m, n = X.shape

# Cost Function
def costFunction(X, Y, theta):
    J = 0
    m = Y.size
    
    # ------- Coding Start -------

    # ------- Coding End -------
       
    return float(J)

#-----------------------------------------------------------
# Evaluation
#-----------------------------------------------------------
def evaluate(J, min, max):
    print("J = ", J, " => Correct") if J > min and J < max else print("J = ", J, " => Wrong")


theta = np.c_[[0, 0, 0]]   
evaluate(costFunction(X, Y, theta), 65591548106.457, 65591548106.458)    
    
theta = np.c_[[30000, 10000, -7000]] 
evaluate(costFunction(X, Y, theta), 55189791032.282, 55189791032.283)

いかがでしょうか。

<h3>続けて課題です。最急降下法(Gradient Descent)のマルチマラメーター対応版も作ってみましょう。</h3>

必要なライブラリは最初にimportしておきましたので、使ってください。<br/>

複数パラメーターが存在する場合の最急降下法の数式は以下の通りです。

$$New \theta_j = Current \theta_j - \alpha * \frac{1}{m}\sum_{i=1}^m(h(X^i) - Y^i)X_j^i$$

<h4>コード作成後、実行すると、自動採点を行います。$Correct$が表示されると成功です。</h4>

<h4>注意：</h4>

- 以下の部分にコードを書いてください。

#------- Coding Start -------
#------- Coding End -------

In [ ]:
# Gradient Descent Function
def gradientDescent(X, Y, theta, alpha = 0.15, iteration = 400):
    m = Y.size
    J_history = np.zeros(iteration)
    Tm = theta
    
    for i in np.arange(iteration):
        # ------- Coding Start -------

        # ------- Coding End -------
        J_history[i] = costFunction(X, Y, Tm)
    
    return Tm, J_history

theta = np.zeros((n,1))
theta, J_history = gradientDescent(X, Y, theta)

#-----------------------------------------------------------
# Evaluation
#-----------------------------------------------------------
def evaluateGD(value, min, max):
    print("J = ", value, " => Correct") if value > min and value < max else print("J = ", value, " => Wrong")
    
print("Cost J Test : ")
evaluateGD(J_history[-1], 0, 2043280050.602829)

plt.plot(J_history)
plt.ylabel(r'J($\theta$)')
plt.xlabel('Iterations');

<h4>全部成功したら、仮定関数の作成は完了です。</h4>

せっかく推論モデルも作りましたので、AIが上手く動作しているかテストしてみましょう。

In [ ]:
# AI Agent to find house price
def AIagent(Xtest, Xmean, Xstd):
    Xnorm = (Xtest - Xmean) / Xstd
    Xnorm = np.hstack((np.ones(1), Xnorm))
    print("House price for %d square feet with %d rooms is %.2f USD."%(Xtest[0], Xtest[1],np.dot(Xnorm, theta)[0]))

# find the price of house with 2000 square feet and 4 rooms
AIagent([2000,4], Xmean, Xstd)    
# AI to find the price of house with 1600 square feet and 2 rooms
AIagent([1600,2], Xmean, Xstd) 

<h4>最後の自分が作った回帰分析コードの結果と、SciKit-learnライブラリが提供するLinearRegression関数の結果も比較してみましょう。</h4>

In [ ]:
# Theta from my Linear regression
print("My theta : ",theta)

# Compare with Scikit-learn Linear regression 
regr = LinearRegression()
regr.fit(X, Y)

print("SciKit-learn's theta : ")
print(regr.intercept_)
print(regr.coef_)